In [1]:
import numpy as np
from numpy import seterr
from numpy import transpose as trans
import sklearn, sys
from sklearn import linear_model
from sklearn.model_selection import KFold
np.seterr(over='raise', under='ignore', divide='raise')

{'divide': 'warn', 'invalid': 'warn', 'over': 'warn', 'under': 'ignore'}

In [2]:
data = np.genfromtxt('data.csv', delimiter = ',')
data = sklearn.preprocessing.normalize(data)
target = np.genfromtxt('targets.csv', delimiter = ',')
data_hat = np.append(data, np.ones([data.shape[0], 1]), 1)

In [13]:
data

array([[  7.92541486e-03,   4.57286305e-03,   5.40989964e-02, ...,
          1.16920795e-04,   2.02695018e-04,   5.23808686e-05],
       [  8.66575595e-03,   7.48616836e-03,   5.59882822e-02, ...,
          7.83583182e-05,   1.15852352e-04,   3.75024596e-05],
       [  9.36668268e-03,   1.01087865e-02,   6.18419883e-02, ...,
          1.15596947e-04,   1.71873157e-04,   4.16624718e-05],
       ..., 
       [  2.14019837e-02,   6.33224349e-02,   1.34594243e-01, ...,
          0.00000000e+00,   5.62134241e-04,   1.58283375e-04],
       [  1.46457050e-02,   2.27665929e-02,   9.67580197e-02, ...,
          1.10921892e-04,   2.26662111e-04,   8.03455949e-05],
       [  1.81413232e-02,   3.77169300e-02,   1.17453641e-01, ...,
          1.52144390e-04,   3.32879884e-04,   1.37628179e-04]])

In [3]:
def sigmoid(x):
    max_elem = max(-x) 
#     print(max_elem)
#     print(np.exp(0 - max_elem))
#     print(np.exp(- x - max_elem))
#     print(min(np.exp(0 - max_elem) + np.exp(- x - max_elem)))
    try: 
        res = -(np.log(np.exp(0 - max_elem) + np.exp(- x - max_elem)) + max_elem ) # Only underflow could happen
        res = np.exp(res)
    except Exception as e:
        res = 0
    return res

In [4]:
def hypo(x_hat, beta):
    return sigmoid(np.matmul(x_hat, beta))

In [5]:
#def cost(x_hat, beta, y):
#    return sum(-np.log(y * np.exp(np.matmul(x_hat, beta)) + 1 - y) + np.log(1 + np.exp(hypo(x_hat, beta)))) 
    # -y*hypo(x_hat, beta) is element-wise
def prob_eq_one(x_hat, beta):
    max_elem = max(np.matmul(x_hat, beta))
    try:
        log =  np.matmul(x_hat, beta) - (np.log(np.exp(0 - max_elem) + np.exp(np.matmul(x_hat, beta) - max_elem)) + max_elem)
        res = trans(np.exp(log))
    except:
        res =  trans(np.zeros([x_hat.shape[0]]))
    # res_2 = np.exp(np.matmul(x_hat, beta)) / (1 + np.exp(np.matmul(x_hat, beta))) # Might Overflow
    # print(res - res_2)
    return res

In [6]:
def error(X, beta, y):
    return np.count_nonzero(abs(np.round(hypo(X, beta)) - y))

In [7]:
def grad(x_hat, beta, y, speed=1):
    p1 = prob_eq_one(x_hat, beta)
    # print(1+np.exp(np.matmul(x_hat, beta)))
    # print(np.exp(np.matmul(x_hat, beta)))
    # print(np.exp(np.matmul(x_hat, beta)) / (1 + np.exp(np.matmul(x_hat, beta))))
    ans = - np.matmul(trans(y) - p1,  x_hat)
    return ans*speed

In [8]:
def hessian(x_hat, beta):
    m, d = x_hat.shape[0], x_hat.shape[1]

    p1 = prob_eq_one(x_hat, beta)
    p1_mult = p1 * (1 - p1)
    res = np.zeros([d, d])
    for i in range(m):
        #print(x_hat[i, :].size)
        #print(trans(x_hat[i, :]).size)
        #print(np.outer(x_hat[i, :], trans(x_hat[i, :])))
        res += p1_mult[i] * np.outer(x_hat[i, :], x_hat[i, :])
    return res

In [9]:
def my_train(X, y, iter, beta):
    for i in range(iter):
        p1 = prob_eq_one(X, beta)
        hess = hessian(X, beta)
        beta_save = beta
        try:
            inv = np.linalg.inv(hess)
            beta -= np.matmul(inv, grad(X, beta, y))
        except Exception as e:
            if(np.max(hess) < np.exp(-100)):
                break
            else:
                beta = beta_save - grad(X, beta, y)
    print(error(data_hat, beta, target))
    return beta

In [10]:
# Train the model with K-Fold
kf = KFold(n_splits=10)
kf.get_n_splits(data)

10

In [11]:
for k, (train, test) in enumerate(kf.split(data, target)):
    m = data_hat[test].shape[0]
    beta = np.zeros(data_hat.shape[1], dtype=float)
    
    iter_beta = my_train(data_hat[train], target[train], 100,  beta)
    
    to_save = np.round(hypo(data_hat[test], iter_beta)).reshape(m, 1)
    to_save = np.dstack((test + 1 , to_save[:,0]))[0]
    np.savetxt('fold%d.csv'%(k+1), to_save, "%d,%d", delimiter=',')

2
6
3
7
4
3
3
4
3
3


In [12]:
iter_beta

array([  1.46562904e+05,  -2.88608022e+03,  -1.78849310e+03,
        -5.07921662e+02,  -1.36886348e+06,   1.61563361e+06,
        -7.61826065e+05,  -9.53858898e+05,   4.20942634e+05,
        -1.34389190e+06,  -5.12949165e+04,   4.59332470e+03,
         2.09013671e+04,  -2.14872872e+03,   2.54039706e+06,
        -2.14127534e+06,   2.06708515e+06,  -9.26396848e+06,
         2.40364346e+06,   1.91349466e+07,  -3.92581263e+04,
        -2.08158551e+03,  -1.95989556e+03,   1.18240520e+03,
         1.90728177e+05,   1.27598129e+05,  -1.76945187e+05,
         6.39076537e+04,  -5.14174441e+05,  -1.78387998e+06,
        -1.20186399e+03])

In [40]:
#Test on all dataset
beta = np.zeros(data_hat.shape[1], dtype=float)
beta = my_train(data_hat, target, 100, beta)

2
